In [ ]:
import json

import pandas as pd

In [7]:
df = pd.read_csv("./edgartools-data.csv")

In [8]:
def process_row(row):
    with open(row["path"], "r", encoding="utf-8") as f:
        file = json.dumps(f.read(), ensure_ascii=False)
    file = file[1:-1]
    # print(f"Processing {row['path']}")
    with open("./predictions.jsonl", "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            line_text = line[5381:]
            if line_text.startswith(file):
                try:
                    line = json.loads(line)
                    line2 = line["response"]["candidates"][0]["content"]["parts"][0][
                        "text"
                    ]
                    out = json.loads(
                        line2,
                        parse_int=str,
                        parse_float=str,
                    )
                    result = pd.Series(
                        {
                            "prediction_idx": i,
                            "chain_of_thought": json.dumps(out["chain_of_thought"]),
                            "estimated_mw": out["estimated_mw"],
                            "score": out["score"],
                            "confidence": out["confidence"],
                        }
                    )
                    return result
                except Exception as e:
                    print(f"Error:{e}\nRow:{row['path']}\nPrediction Index:{i}")
                    return pd.Series(
                        {
                            "prediction_idx": i,
                            "chain_of_thought": None,
                            "estimated_mw": None,
                            "score": None,
                            "confidence": None,
                        }
                    )
        print(f"Error:No matching prediction found\nRow:{row['path']}")
        return pd.Series(
            {
                "prediction_idx": None,
                "chain_of_thought": None,
                "estimated_mw": None,
                "score": None,
                "confidence": None,
            }
        )

In [9]:
df[["prediction_idx", "chain_of_thought", "estimated_mw", "score", "confidence"]] = (
    df.apply(process_row, axis=1)
)

Error:Expecting ',' delimiter: line 3 column 63642 (char 65410)
Row:./edgartools-data/CLSK/8-K/2022-08-23.md
Prediction Index:921


In [10]:
df.to_csv("./predictions.csv", index=False)

In [13]:
df.head().to_csv("./predictions-head.csv", index=False)